# SFINCS-GFM

In this notebook we create and run a workflow for running the Sentinel-1 Global Flood Monitor and a SFINCS model for a flood event and comparing their outputs. We use the SFINCS model created in the `create_sfincs.ipynb` notebook and update it with the relevant forcing as part of the workflow. To run the SFINCS model, the workflow will trigger an OSCAR service to run the model remotely. For this the proper credentials are necessary. We use a refresh token generated from EGI SSO's Check-in portal. Make sure that the user is part of the dev.intertwin.eu VO.

In [ ]:
from pathlib import Path

from hydroflows import Workflow, WorkflowConfig
from hydroflows.log import setuplog

from sfincs_gfm.methods import gfm, breach_dikes, set_forcing, run_oscar, comparison_sfincs_gfm

logger = setuplog(level="INFO")

## Basic Config

Here we set some basic parameters of the workflow. These are:
- `region`: file containing region boundaries used by GFM
- `start_date`: event start date
- `end_date`: event end data
- `sf_root`: root folder of the SFINCS model
- `sfincs_service`: OSCAR service used to run the SFINCS model
- `refreshtoken`: EGI SSO refresh token for authentication for OSCAR

The workflow itself will be created as the `wf` object. The root folder for workflow execution is set to the current folder, and all paths defined in this notebook are relative to that root.

In [ ]:
config = WorkflowConfig(
    region=Path("data", "region.geojson"),
    start_date="2023-10-15",
    end_date="2023-10-23",
    sf_root=Path("models/20m_final"),
    sfincs_service=Path("oscar_services/sfincs-interlink.yaml"),
    refreshtoken=""
)

In [ ]:
wf = Workflow(name="baltic_gfm", config=config, root=Path().resolve())

## GFM

Here we add a step to the workflow to run the GFM. Actual execution of GFM will happen only later when we call `wf.run()` to launch the entire workflow.\
The important additional parameter here is `output_dir` which defines where the GFM output will be stored.

In [ ]:
run_gfm = gfm.GFM(
    region=wf.config.region,
    starttime=wf.config.start_date,
    endtime=wf.config.end_date,
    output_dir=Path("gfm"),
)

wf.create_rule(run_gfm, rule_id="run_gfm")

## Updating SFINCS model

The next two cells add steps to the workflow for updating the SFINCS model with the right forcing (1st cell) and the right dike breach locations (2nd cell). These are separated from each other so that the number of events and breach scenarios run as part of the workflow can vary independently. The important parameters are:

#### SetForcing:
- `output_dir`: location where the updated model will be stored.
- `wl_path`: path to file with waterlevel forcing. Here we use data from a local source prepared ahead of time.
- `wind_url`: URL where to fetch the wind data from. Unlike the waterlevel data, the wind data is fetched on the fly.

#### BreachDikes:
- `output_dir`: location where the updated model will be stored
- `dike_locs`: path to file with all the dikes used in the initial model setup
- `breach_locs`: path to file with the breach locations

In [ ]:
set_forcing = set_forcing.SetForcing(
    sfincs_inp=wf.config.sf_root/"sfincs.inp",
    output_dir="models/runs/test_run",
    copy_model=True,
    start_time=wf.config.start_date,
    end_time=wf.config.end_date,
    wl_file=Path("data\waterlevel\pegelonline-barth-W-20231015-20231023.csv"),
    wind_url="https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/wind/historical/stundenwerte_FF_00298_19810101_20231231_hist.zip"
)

wf.create_rule(set_forcing, rule_id="set_forcing")

In [ ]:
breach_dikes = breach_dikes.BreachDikes(
    sfincs_inp=set_forcing.output.sfincs_out_inp,
    dike_locs=Path("data/dikes/all_dikes.gpkg"),
    breach_locs=Path("data/dikes/test_breaches.geojson"),
    output_dir="models/runs/test_run_breach",
    copy_model=True,
)

wf.create_rule(breach_dikes, rule_id="breach_dikes")

## Run the SFINCS model

The next cell adds the step to trigger the OSCAR service and run the SFINCS model. This will use the OSCAR service and access token defined in the `config`

In [ ]:
run_oscar = run_oscar.RunOscarService(
    input_file=breach_dikes.output.sfincs_out_inp,
    service=wf.config.sfincs_service,
    refreshtoken=wf.config.refreshtoken,
)

wf.create_rule(run_oscar, rule_id="run_oscar_breach")

## Compare SFINCS and GFM

Finally we add the step to do a comparison of the SFINCS floodmap and the GFM output.\
NB: this is still under development, and this step currently does nothing.

In [ ]:
compare = comparison_sfincs_gfm.CompareSfincsGFM(
    sfincs_inp=run_oscar.output.output_file,
    gfm_map=run_gfm.output.floodcube,
    output_dir=Path("final_out")
)

wf.create_rule(compare, rule_id="compare_sfincs_gfm")

## Inspect workflow

To get a sense of what the workflow will be doing before actually executing it, the next cell plots the execution graph of the workflow.\
The cell after that will do a dryrun of the workflow, where empty files are created at the output locations without doing the full execution of each step. This is useful to double check whether the workflow does what it is supposed to be doing.

In [ ]:
wf.plot_rulegraph(plot_rule_attrs=True)

In [ ]:
wf.dryrun()

## Execute workflow

In [ ]:
wf.run()